In [1]:
import torch
import clip
from PIL import Image

In [7]:
torch.set_grad_enabled(False)

In [2]:
device = "cuda"

class ImageLoader:
    def __init__(self, device, preprocessor):
        self.device = device
        self.preprocessor = preprocessor
    
    def __call__(self, filepath):
        img = Image.open(filepath)
        return self.preprocessor(img).unsqueeze(0).to(self.device)

loader = ImageLoader(device, preprocess)
model, preprocess = clip.load("ViT-B/32", device=device)

In [38]:
text = clip.tokenize(["picture", "of", "a building", "a computer", "a lake", "a mountain"]).to(device)
img = loader("images/lake.jpg")

In [39]:
image_features = model.encode_image(img)
text_features = model.encode_text(text)

In [40]:
image_features.shape

torch.Size([1, 512])

In [41]:
text_features.shape

torch.Size([6, 512])

In [42]:
text_features.norm(dim=-1, keepdim=True)

tensor([[13.2344],
        [12.7578],
        [10.7422],
        [10.4141],
        [10.1094],
        [10.5000]], device='cuda:0', dtype=torch.float16)

In [43]:
# @ performs matrix multiplication
# Cosine similarity is equivalent of dot product after normalization of embeddings

# Normalize all vectors, for text every vector is normalized separately.
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

In [62]:
# Compute cosine similarity between image and all text embeddings
similarity = 100 * image_features @ text_features.T

In [63]:
similarity.softmax(dim=-1)

tensor([[5.8289e-03, 2.1782e-03, 8.0585e-05, 2.1565e-04, 9.3555e-01, 5.6183e-02]],
       device='cuda:0', dtype=torch.float16)